<a href="https://colab.research.google.com/github/C0ntinue/EEE4171_AICom/blob/main/project3_20181485_20181573_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#pj_root= '/content/gdrive/MyDrive/서강대 학습자료/2021년도 1학기 학습자료/인공지능통신-김홍석 교수님/프로젝트3/'
pj_root = '/content/gdrive/MyDrive/Data/서강전자7학기/인공지능통신/pj3/data/'

In [3]:
# import dependency
import os
import numpy as np
import matplotlib as plt
import pandas as pd

In [4]:
# upload data (train_csv, test_csv, test_label_csv)
# 1st col : data number
# 2nd col : userId
# 3rd col : movieId
# 4th col : rating

# train_csv = pd.read_csv(pj_root+"movie_train.csv", index_col = 0)
# test_csv = pd.read_csv(pj_root+"movie_test.csv", index_col = 0)
# test_label_csv = pd.read_csv(pj_root+"movie_test_label.csv", index_col = 0)

train_csv = pd.read_csv(pj_root+"example_train.csv", index_col = 0)
test_csv = pd.read_csv(pj_root+"example_test.csv", index_col = 0)
test_label_csv = pd.read_csv(pj_root+"example_test_label.csv", index_col = 0)

In [5]:
train_csv

,userId,movieId,rating
1,1,1,5
2,1,2,4
3,1,3,4
4,2,2,3
5,2,3,5
6,2,5,4
7,3,1,5
8,3,2,2
9,3,5,3
10,4,3,3


In [6]:
train_rating = train_csv.pivot_table('rating', index = 'userId', columns = 'movieId').fillna('')
train_rating

movieId,1,2,3,4,5
userId,,,,,
1,5,4,4,,
2,,3,5,,4
3,5,2,,,3
4,,,3,1,2
5,4,,,4,5
6,,3,,3,5
7,3,,3,2,
8,5,,4,,5
9,,2,5,4,


In [7]:
test_csv

,userId,movieId,rating
1,1,5,0
2,2,4,0
3,3,4,0
4,4,2,0
5,5,3,0
6,6,1,0
7,7,2,0
8,8,2,0
9,9,1,0
10,10,1,0


In [8]:
test_rating = test_csv.pivot_table('rating', index = 'userId', columns = 'movieId').fillna('')
test_rating

movieId,1,2,3,4,5
userId,,,,,
1,,,,,0
2,,,,0,
3,,,,0,
4,,0,,,
5,,,0,,
6,0,,,,
7,,0,,,
8,,0,,,
9,0,,,,


In [9]:
test_label_csv

,userId,movieId,rating
1,1,5,5
2,2,4,3
3,3,4,2
4,4,2,2
5,5,3,5
6,6,1,5
7,7,2,2
8,8,2,3
9,9,1,4
10,10,1,5


In [10]:
test_label_rating = test_label_csv.pivot_table('rating', index = 'userId', columns = 'movieId').fillna('')
test_label_rating

movieId,1,2,3,4,5
userId,,,,,
1,,,,,5
2,,,,3,
3,,,,2,
4,,2,,,
5,,,5,,
6,5,,,,
7,,2,,,
8,,3,,,
9,4,,,,


# Train

## Train baseline predictor

In [15]:
# r의 평균 구하기
sum = 0.0
count = 0
for i in train_rating:
  for val in train_rating[i]:
    if val != '':
      sum = sum + val
      count = count + 1
r_avg = sum/count
r_avg

3.6666666666666665

In [16]:
train_rating_values=train_rating.values
train_rating_values

array([[5.0, 4.0, 4.0, '', ''],
       ['', 3.0, 5.0, '', 4.0],
       [5.0, 2.0, '', '', 3.0],
       ['', '', 3.0, 1.0, 2.0],
       [4.0, '', '', 4.0, 5.0],
       ['', 3.0, '', 3.0, 5.0],
       [3.0, '', 3.0, 2.0, ''],
       [5.0, '', 4.0, '', 5.0],
       ['', 2.0, 5.0, 4.0, ''],
       ['', '', 5.0, 3.0, 4.0]], dtype=object)

In [17]:
# A, c 구성
num_users = len(train_rating.index)
num_movies = len(train_rating.columns)

A = np.zeros((count,num_users + num_movies))
cT = np.zeros(count)
data_idx = 0
for user_id,ratings in enumerate(train_rating_values):
  for movie_id,val in enumerate(ratings):
    if val != '':
      A[data_idx][user_id]=1
      A[data_idx][num_users+movie_id]=1
      cT[data_idx]=val-r_avg
      data_idx = data_idx +1

c=np.transpose(cT)
print("\n matrix A의 값:")
print(A)
print("\n vector c의 값:")
print(c)



 matrix A의 값:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.

In [18]:
# pseudo inverse로 최적의 b 구하기
bT = np.dot(np.linalg.pinv(A),c)
b = np.transpose(bT)

print("\n vector b의 값:")
print(b)


 vector b의 값:
[ 0.62592593  0.42348485 -0.28148148 -1.77474747  0.5202862   0.49250842
 -1.23897306  0.45126263  0.39991582  0.22525253  0.71792929 -1.19873737
  0.6030303  -0.6040404   0.32525253]


## Obtain baseline prediction

In [19]:
#baseline prediction
test_rating_values = test_rating.values 
P1 = np.zeros(np.shape(test_rating_values))
for user_id,ratings in enumerate(test_rating_values):
  for movie_id,val in enumerate(ratings):
    if val == 0:
      P1[user_id][movie_id] = (r_avg+b[user_id]+b[num_users+movie_id])


print("\n baseline prediction matrix P1의 값:")
print(P1)


 baseline prediction matrix P1의 값:
[[0.         0.         0.         0.         4.61784512]
 [0.         0.         0.         3.48611111 0.        ]
 [0.         0.         0.         2.78114478 0.        ]
 [0.         0.69318182 0.         0.         0.        ]
 [0.         0.         4.78998316 0.         0.        ]
 [4.87710438 0.         0.         0.         0.        ]
 [0.         1.22895623 0.         0.         0.        ]
 [0.         2.91919192 0.         0.         0.        ]
 [4.78451178 0.         0.         0.         0.        ]
 [4.60984848 0.         0.         0.         0.        ]]


In [20]:
# error matrix 구성
R_tilde = np.zeros(np.shape(train_rating_values))
R_tilde_zeros = np.zeros(np.shape(train_rating_values))

for user_id,ratings in enumerate(train_rating_values):
  for movie_id,val in enumerate(ratings):
    if val != '':
      R_tilde[user_id][movie_id] = train_rating_values[user_id][movie_id]-(r_avg+b[user_id]+b[num_users+movie_id])
      R_tilde_zeros[user_id][movie_id] = train_rating_values[user_id][movie_id]-(r_avg+b[user_id]+b[num_users+movie_id])
    else:
       R_tilde[user_id][movie_id] = None

print("baseline prediction matrix R_tilde: ")
print(np.round(R_tilde,2))
print()
print(R_tilde)

baseline prediction matrix R_tilde: 
[[-0.01  0.91 -0.9    nan   nan]
 [  nan  0.11  0.31   nan -0.42]
 [ 0.9  -0.19   nan   nan -0.71]
 [  nan   nan  0.51 -0.29 -0.22]
 [-0.9    nan   nan  0.42  0.49]
 [  nan  0.04   nan -0.56  0.52]
 [-0.15   nan -0.03  0.18   nan]
 [ 0.16   nan -0.72   nan  0.56]
 [  nan -0.87  0.33  0.54   nan]
 [  nan   nan  0.51 -0.29 -0.22]]

[[-0.01052189  0.90614478 -0.8956229          nan         nan]
 [        nan  0.10858586  0.30681818         nan -0.41540404]
 [ 0.89688552 -0.18644781         nan         nan -0.71043771]
 [        nan         nan  0.50505051 -0.28787879 -0.21717172]
 [-0.90488215         nan         nan  0.41708754  0.48779461]
 [        nan  0.03956229         nan -0.55513468  0.51557239]
 [-0.1456229          nan -0.03072391  0.1763468          nan]
 [ 0.16414141         nan -0.7209596          nan  0.55681818]
 [        nan -0.86784512  0.33038721  0.53745791         nan]
 [        nan         nan  0.50505051 -0.28787879 -0.21717172]]


## Compute similarity matrix

In [21]:
# cosine coefficient 구하는 함수

def cosine_similarity(v1,v2):
    # "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    np.nan_to_num(v1, copy=False)
    np.nan_to_num(v2, copy=False)
    valid_idx = np.logical_and(v1,v2)
    for i in range(len(v1)):
      if valid_idx[i] == True:
        x =v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/np.sqrt(sumxx*sumyy)

In [22]:
# similarity matrix 구성
num_movies = len(train_rating.columns)
sim_matrix = np.zeros((num_movies,num_movies))
train_ratingsT = np.transpose(R_tilde)
for i in range(num_movies):
  for j in range(num_movies):
    if i == j:
      sim_matrix[i][j] = 0
    else:
      sim_matrix[i][j] = cosine_similarity(train_ratingsT[i],train_ratingsT[j])

print()
print('반올림한 similarity matrix의 값:')
print(np.round(sim_matrix,2))

print()
print('similarity matrix의 실제 값:')
print(sim_matrix)


반올림한 similarity matrix의 값:
[[ 0.   -0.21 -0.41 -0.97 -0.75]
 [-0.21  0.   -0.84 -0.73  0.51]
 [-0.41 -0.84  0.   -0.22 -0.93]
 [-0.97 -0.73 -0.22  0.    0.07]
 [-0.75  0.51 -0.93  0.07  0.  ]]

similarity matrix의 실제 값:
[[ 0.         -0.21301359 -0.41335723 -0.97123137 -0.74899568]
 [-0.21301359  0.         -0.84334288 -0.72757202  0.505802  ]
 [-0.41335723 -0.84334288  0.         -0.21614403 -0.92916674]
 [-0.97123137 -0.72757202 -0.21614403  0.          0.06791977]
 [-0.74899568  0.505802   -0.92916674  0.06791977  0.        ]]


## Define neighborhood

In [23]:
# neighborhood 개수 정하기
L = 2

## Obtain neghborhood prediction

In [24]:
# neighborhood predictor matrix 구성
test_rating_values = test_rating.values
P2 = np.zeros(np.shape(P1))
for user_id,ratings in enumerate(test_rating_values):
  for movie_id,val in enumerate(ratings):
    if val == 0:
      abs_rev_sorted_sim_list = list(reversed(sorted(map(abs,sim_matrix[movie_id]))))
      valid_sim = np.logical_and(train_rating_values[user_id],sim_matrix[movie_id])
      for i,sim in enumerate(valid_sim):
        if valid_sim[i] == '':
          valid_sim[i] = 0 
      q1 = 0
      q2 = 0
      for i in range(L):
        movie_idx = list(abs(valid_sim)).index(abs_rev_sorted_sim_list[i]) if abs_rev_sorted_sim_list[i] in list(abs(valid_sim)) else None
        if movie_idx is not None:
          q2 += sim_matrix[movie_id][movie_idx]*R_tilde[user_id][movie_idx]
          q1 += abs(sim_matrix[movie_id][movie_idx])
      if q1 != 0 and q2 != 0:
        P2[user_id][movie_id]+=  P1[user_id][movie_id]+q2/q1
      else:
        P2[user_id][movie_id]+=  P1[user_id][movie_id]
print(P2)

[[0.         0.         0.         0.         5.11843066]
 [0.         0.         0.         3.37752525 0.        ]
 [0.         0.         0.         2.34823468 0.        ]
 [0.         0.55537792 0.         0.         0.        ]
 [0.         0.         4.30218855 0.         0.        ]
 [4.96604771 0.         0.         0.         0.        ]
 [0.         1.16377497 0.         0.         0.        ]
 [0.         3.64015152 0.         0.         0.        ]
 [4.24705387 0.         0.         0.         0.        ]
 [4.86694105 0.         0.         0.         0.        ]]


# Test

In [31]:
def rmse(predictions, targets):
  sse = 0
  cnt = 0
  for i in range(len(predictions)):
    for j in range(len(predictions[0])):
      if targets[i][j] != '':
        sse += (predictions[i][j]-targets[i][j])**2
        cnt+=1
  return np.sqrt(sse/cnt)

In [26]:
def acc(predictions, targets):
  err = 0
  sot = 0
  cnt = 0
  for i in range(len(predictions)):
    for j in range(len(predictions[0])):
      if targets[i][j] != '':
        err += abs(predictions[i][j]-targets[i][j])/targets[i][j]
        cnt+=1
  return err*100/cnt

In [32]:
# RMSE와 accuracy 구하기
baseline_rmse = rmse(P1,test_label_rating.values)
baseline_acc = acc(P1,test_label_rating.values)

nei_rmse = rmse(P2,test_label_rating.values)
nei_acc = acc(P2,test_label_rating.values)

In [28]:
print(baseline_rmse)
print(baseline_acc)
print(nei_rmse)
print(nei_acc)

0.6425132752294196
20.356481481481463
0.6356612009512063
19.121806238243565


In [35]:
clipped_P2 = P2
clipped_P2[3,1] = 1
clipped_P2[0,4] = 5
print(rmse(clipped_P2,test_label_rating.values))

0.5421893440091285
